# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 5e5 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 1000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "ShooterTraining" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 128 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'ML_Academy' started successfully!


Unity Academy name: ML_Academy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: ML_Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 6
        Action space type: continuous
        Action space size (per agent): 3
        Memory space size (per agent): 0
        Action descriptions: , , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 1000. Mean Reward: 0.015567784021978025. Std of Reward: 0.07871354572624693.
Step: 2000. Mean Reward: 0.031225210549450547. Std of Reward: 0.11784528979081212.
Step: 3000. Mean Reward: 0.017971876. Std of Reward: 0.10276059769357078.
Step: 4000. Mean Reward: 0.015435601648351647. Std of Reward: 0.10981987539419287.
Step: 5000. Mean Reward: 0.020366247604395604. Std of Reward: 0.08368696664927823.
Step: 6000. Mean Reward: 0.025320845824175823. Std of Reward: 0.12086939237128859.
Step: 7000. Mean Reward: 0.030257152857142856. Std of Reward: 0.12407649581668773.
Step: 8000. Mean Reward: 0.008058921494505496. Std of Reward: 0.08854430319871574.
Step: 9000. Mean Reward: 0.0008822545494505497. Std of Reward: 0.08703767496511025.
Step: 10000. Mean Reward: 0.017778587032608695. Std of Reward: 0.10222287178450622.
Step: 11000. Mean Reward: 0.015062948373626374. Std of Reward: 0.08905676862226533.
Step: 12000. Mean Reward: 0.018359841076923074. Std of Reward: 0.11275441973949993.
Step: 130

Step: 99000. Mean Reward: 0.007764950054945056. Std of Reward: 0.09835696607841458.
Step: 100000. Mean Reward: 0.019872336912087913. Std of Reward: 0.08243563482413017.
Saved Model
Step: 101000. Mean Reward: 0.0007492284725274736. Std of Reward: 0.07463625820179039.
Step: 102000. Mean Reward: -0.001487153351648351. Std of Reward: 0.07173878418319844.
Step: 103000. Mean Reward: 0.0018780080549450553. Std of Reward: 0.06443644597162859.
Step: 104000. Mean Reward: 0.0024853632747252746. Std of Reward: 0.0678625782309059.
Step: 105000. Mean Reward: 0.007484179274725274. Std of Reward: 0.04993487661137589.
Step: 106000. Mean Reward: 0.007912634428571428. Std of Reward: 0.07636789071830291.
Step: 107000. Mean Reward: 0.021809808956043952. Std of Reward: 0.0883956747390487.
Step: 108000. Mean Reward: 0.0012077709340659343. Std of Reward: 0.0818634610089978.
Step: 109000. Mean Reward: 0.005860791141304348. Std of Reward: 0.0867997300085181.
Step: 110000. Mean Reward: -0.0008745508791208788. St

Step: 196000. Mean Reward: 0.003337251384615385. Std of Reward: 0.07103790526605412.
Step: 197000. Mean Reward: 0.01243499041304348. Std of Reward: 0.07349802606067911.
Step: 198000. Mean Reward: 0.02258063675824176. Std of Reward: 0.1106234010644842.
Step: 199000. Mean Reward: 0.01129953132967033. Std of Reward: 0.06978007220541531.
Step: 200000. Mean Reward: 0.012174693153846155. Std of Reward: 0.057778919697198144.
Saved Model
Step: 201000. Mean Reward: 0.011107252626373625. Std of Reward: 0.08297647174401757.
Step: 202000. Mean Reward: 0.017423349494505497. Std of Reward: 0.07998397118035834.
Step: 203000. Mean Reward: 0.011121990417582418. Std of Reward: 0.06935564185302055.
Step: 204000. Mean Reward: 0.002167205989010989. Std of Reward: 0.06614458859730193.
Step: 205000. Mean Reward: 0.01343326110989011. Std of Reward: 0.08427856822275956.
Step: 206000. Mean Reward: 0.00546187078021978. Std of Reward: 0.05843433842415536.
Step: 207000. Mean Reward: 0.026323439791208787. Std of Re

Step: 293000. Mean Reward: 0.019148979747252746. Std of Reward: 0.09712702152068953.
Step: 294000. Mean Reward: 0.002851515934065934. Std of Reward: 0.0692010680448225.
Step: 295000. Mean Reward: 0.019423067769230768. Std of Reward: 0.0978423392639491.
Step: 296000. Mean Reward: 0.014075374945652174. Std of Reward: 0.09110135679490382.
Step: 297000. Mean Reward: 0.02726683365934066. Std of Reward: 0.10954361418171871.
Step: 298000. Mean Reward: 0.014780207340659341. Std of Reward: 0.08414967125363204.
Step: 299000. Mean Reward: 0.015113801087912089. Std of Reward: 0.06431916271536448.
Step: 300000. Mean Reward: 0.017453107428571424. Std of Reward: 0.09719590541644414.
Saved Model
Step: 301000. Mean Reward: 0.01464105357142857. Std of Reward: 0.0688693396647103.
Step: 302000. Mean Reward: 0.02680176591208791. Std of Reward: 0.09953889900102225.
Step: 303000. Mean Reward: 0.01312203879120879. Std of Reward: 0.09027419338632829.
Step: 304000. Mean Reward: 0.010029006439560438. Std of Rewa

Step: 390000. Mean Reward: 0.03191157890109889. Std of Reward: 0.11596928271959278.
Step: 391000. Mean Reward: 0.00740635834065934. Std of Reward: 0.06744281716582522.
Step: 392000. Mean Reward: 0.023000746846153847. Std of Reward: 0.09829904007367427.
Step: 393000. Mean Reward: 0.03039290556043956. Std of Reward: 0.11333827675920102.
Step: 394000. Mean Reward: 0.013410180692307691. Std of Reward: 0.10074335592248158.
Step: 395000. Mean Reward: 0.010196547119565217. Std of Reward: 0.07281291349509672.
Step: 396000. Mean Reward: 0.01684631665934066. Std of Reward: 0.10512595813559124.
Step: 397000. Mean Reward: 0.012295233351648352. Std of Reward: 0.06668636467450638.
Step: 398000. Mean Reward: 0.020344991142857143. Std of Reward: 0.10071515440842739.
Step: 399000. Mean Reward: 0.02255136778021978. Std of Reward: 0.10090288353102284.
Step: 400000. Mean Reward: 0.014194213780219778. Std of Reward: 0.07580771172739885.
Saved Model
Step: 401000. Mean Reward: 0.01442968623076923. Std of Rew

Step: 487000. Mean Reward: 0.014482521846153848. Std of Reward: 0.09348947990290714.
Step: 488000. Mean Reward: 0.033115092736263736. Std of Reward: 0.1221860064077592.
Step: 489000. Mean Reward: 0.0037837923736263736. Std of Reward: 0.0731310350959136.
Step: 490000. Mean Reward: 0.012055389637362638. Std of Reward: 0.08003982867970118.
Step: 491000. Mean Reward: 0.03079188512087912. Std of Reward: 0.12120834267080756.
Step: 492000. Mean Reward: 0.03405768275824175. Std of Reward: 0.11697728842877678.
Step: 493000. Mean Reward: 0.0094971420989011. Std of Reward: 0.06335432655161854.
Step: 494000. Mean Reward: 0.02201002231521739. Std of Reward: 0.09070163228033978.
Step: 495000. Mean Reward: 0.012437610087912087. Std of Reward: 0.0734765581606049.
Step: 496000. Mean Reward: 0.01301485857142857. Std of Reward: 0.09860491377029541.
Step: 497000. Mean Reward: 0.03163273432967033. Std of Reward: 0.11552130450413477.
Step: 498000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 499000. Mean Rew

INFO:tensorflow:Restoring parameters from ./models/ppo\model-500001.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.


### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-500001.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-500001.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.
